# Notebook 3: Dataset Integration

## README

### Overview

This notebook integrates spatial NO₂ data with multiple auxiliary datasets to generate a unified, mesh-based dataset for modeling and analysis in two case study cities: **Addis Ababa, Ethiopia** and **Baghdad, Iraq**.  

It contains two part: (1) multi-dataset integration & new feature generation, and (2) data temporal & spacial aggregation.

It consolidates outputs from:

- **NO₂ pollution data** (processed in Notebook 1)  
- **Human activity and other environmental feature datasets** (processed in Notebook 2)

### Objective

- **Merge**: Join NO₂ pollution values with auxiliary features (e.g., population, roads, night lights) by mesh ID and date.  
- **New Feature**: Generate derived features such as NO2 lag feature and No2 neighor lag featrue.
- **Aggregation**: Get prepared dataset with different temporal-spatial resolution.


### Workflow

1. **Feature Merging**  
   - Load cleaned NO₂ and feature datasets from previous notebooks  
   - Merge all spatial datasets by mesh cell (`geom_id`)  
   - Ensure alignment in spatial extent and resolution  

2. **Feature Engineering**  
   - Compute new features (e.g., spatial & temporal-based new feature)  

3.  **Aggregation and Export**  
   - Aggregate in time: from daily value to monthly level.
   - Aggregate in space: from cell level to sub-administrative region.
   - Export all the data in progress as GeoPackage or CSV for downstream tasks  


### Outputs

- `city_mesh_data.gpkg`: GeoDataFrame with all integrated features per mesh cell, one file per day (731 files for each city).
- `full_city_df.parquet`: Concatenated DataFrame including all dates in one file, convenient for downstream analysis (one file for each city).
- `city_timeResolution_spaceResolution.csv`: Aggregated DataFrame including all dates in one file, in differen data resolution.


## 0 Init: Prepare Packages and Configuration

Get current file/repo/data path in local to make sure the following cells run properly.

In [ ]:
import sys
from pathlib import Path
SRC_PATH = Path().resolve().parent / "src"
sys.path.append(str(SRC_PATH))

from config import *

## 1 Merge Multiple Features

Combine features from different GeoPackage files into one GeoPackage file. 

Then save the data as parquet format for better reading efficiency.

### Addis Ababa

In [ ]:
# Addis Ababa
from helpercollections import merge_multiple_gpkgs, convert_gpkgs_to_parquet
import geopandas as gpd

addis_feature_mesh_paths = [DATA_PATH / "addis-no2-mesh-data",      # NO2 data: numeric
                            DATA_PATH / "addis-temp-mesh-data",     # Daily Average Temperature 
                            DATA_PATH / "addis-pop-mesh-data",      # Population data: numeric
                            DATA_PATH / "addis-NTL-mesh-data",      # Night Time Light data: numeric
                            DATA_PATH / "addis-CC-mesh-data",       # Cloud Category data: category
                            DATA_PATH / "addis-LST-mesh-data",      # Land Surface Temperature data: numeric
                            DATA_PATH / "addis-lc-mesh-data",       # Land Cover data: category
                            DATA_PATH / "addis-ESA-mesh-data",      # Land Cover ESA (area) data: numeric
                            DATA_PATH / "addis-OSM-mesh-data",      # Point Of Interest data: numeric
                            DATA_PATH / "addis-OSM2-mesh-data",     # New Land Use Data, combined with ESA and OSM
                            ]
output_folder = DATA_PATH / "addis-mesh-data"
merge_multiple_gpkgs(addis_feature_mesh_paths, output_folder)
convert_gpkgs_to_parquet(mesh_folder=output_folder, output_path=DATA_PATH / "temp", file_name="full_addis_df") 

In [ ]:
gdf_addis = gpd.read_file(DATA_PATH / "addis-mesh-data" / "addis-ababa-2023-01-02.gpkg")
gdf_addis.head(3)

,geom_id,no2_mean,temp_mean,pop_sum_m,NTL_mean,cloud_category,LST_day_mean,landcover_2023,tree_cover_a,shrubland_a,...,lu_farmyard_share,road_motorway_len,road_trunk_len,road_primary_len,road_secondary_len,road_tertiary_len,road_residential_len,fossil_pp_count,non_built_area,geometry
0,0,0.00005,25.13,969.683960,10.372897,0.0,27.732857,12.0,9900,113700,...,0.0,0.0,0.0,0.0,0.0,1351.258983,3079.516097,0,198200,"POLYGON ((38.78925 8.83942, 38.78925 8.84841, ..."
1,1,0.00005,22.73,1574.853149,10.460668,1.0,27.150000,12.0,200,80600,...,0.0,0.0,0.0,0.0,0.0,0.000000,3690.168402,0,149900,"POLYGON ((38.79824 8.83942, 38.79824 8.84841, ..."
2,2,0.00005,21.05,1859.865723,10.903213,1.0,27.150000,13.0,33300,40500,...,0.0,0.0,0.0,0.0,0.0,846.632771,2949.800017,0,187500,"POLYGON ((38.80722 8.83942, 38.80722 8.84841, ..."


### Iraq - Baghdad

In [ ]:
# # Baghdad
from helpercollections import merge_multiple_gpkgs, convert_gpkgs_to_parquet
import geopandas as gpd

baghdad_feature_mesh_paths = [DATA_PATH / "baghdad-no2-mesh-data",      # NO2 data: numeric
                              DATA_PATH / "baghdad-pop-mesh-data",      # Population data: numeric
                              DATA_PATH / "baghdad-NTL-mesh-data",      # Night Time Light data: numeric
                              DATA_PATH / "baghdad-CC-mesh-data",       # Cloud Category data: category
                              DATA_PATH / "baghdad-LST-mesh-data",      # Land Surface Temperature data: numeric
                              DATA_PATH / "baghdad-temp-mesh-data",     # Daily Average Temperature 
                              DATA_PATH / "baghdad-lc-mesh-data",       # Land Cover data: category
                              DATA_PATH / "baghdad-ESA-mesh-data",      # Land Cover ESA (area) data: numeric
                              DATA_PATH / "baghdad-OSM-mesh-data",      # Point Of Interest data: numeric
                              DATA_PATH / "baghdad-TCI-mesh-data",      # Traffic Congestion Index data: numeric
                              DATA_PATH / "baghdad-OSM2-mesh-data",     # New Land Use Data, combined with ESA and OSM
                              ]
  
output_folder = DATA_PATH / "baghdad-mesh-data"
merge_multiple_gpkgs(baghdad_feature_mesh_paths, output_folder)
convert_gpkgs_to_parquet(mesh_folder=output_folder, output_path=DATA_PATH / "temp", file_name="full_baghdad_df") 

In [ ]:
gdf_baghdad = gpd.read_file(DATA_PATH / "baghdad-mesh-data" / "baghdad-2023-01-01.gpkg")
gdf_baghdad.head(3)

,geom_id,no2_mean,pop_sum_m,NTL_mean,cloud_category,LST_day_mean,temp_mean,landcover_2023,tree_cover_a,shrubland_a,...,road_motorway_len,road_trunk_len,road_primary_len,road_secondary_len,road_tertiary_len,road_residential_len,fossil_pp_count,TCI,non_built_area,geometry
0,0,0.000111,44.653709,2.088287,0.0,18.09,11.831877,12.0,1400,35500,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,333800,"POLYGON ((44.56542 32.81546, 44.56542 32.82444..."
1,1,0.000111,45.872368,2.345411,0.0,17.79,11.828498,12.0,3300,43200,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,613300,"POLYGON ((44.5744 32.81546, 44.5744 32.82444, ..."
2,2,0.000111,40.733757,2.524289,0.0,17.75,11.824177,10.0,21700,65400,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,788800,"POLYGON ((44.58339 32.81546, 44.58339 32.82444..."


## 2 Generate New Features 

Based on the exploratory data analysis below, generate new features related to NO₂ spatial and temporal autocorrelation.

The following cell made the pre-processing of the data, including:

1) Integrate all the gpkg into a large file with date.

2) Add new features:

    - Yesterday NO₂ (1 day lag)

    - Yesterday average NO₂ level.

3) Export as parquet file for faster read in the future.

### Addis Ababa

In [ ]:
import pandas as pd
from analysis import add_lag_features
full_addis_df = pd.read_parquet(DATA_PATH / "temp" / "full_addis_df.parquet", engine="pyarrow")

add_lag_features(full_addis_df, output_path=DATA_PATH / "temp", file_name="full_addis_df")

### Baghdad

In [ ]:
import pandas as pd
from analysis import add_lag_features
full_baghdad_df = pd.read_parquet(DATA_PATH / "temp" / "full_baghdad_df.parquet", engine="pyarrow")

add_lag_features(full_baghdad_df, output_path=DATA_PATH / "temp", file_name="full_baghdad_df")

## 3 Aggregation: Temporal & Spatial Aggregation

- Initial data resolution:
    - Time: daily
    - Space: cell (mesh grid)

- Total four types of data with different aggregation level:
    1. Cell level & Daily Data (original parquet file)
    2. Cell level & Monthly Data
    3. Sub-region level & Daily Data
    4. Sub-region level & Monthly Data

### Addis Ababa

In [ ]:
import numpy as np
import pandas as pd
# Read the Addis Ababa data
full_df = pd.read_parquet(DATA_PATH / "temp" / "full_addis_df.parquet", engine="pyarrow")
full_df['month'] = full_df['date'].dt.to_period('M')

# Original dataset: daily / cell resolution
daily_cell_ori = full_df

In [ ]:
# type(full_df)
full_df.columns

Index(['geom_id', 'no2_mean', 'pop_sum_m', 'NTL_mean', 'road_len',
       'road_share', 'poi_count', 'poi_share', 'lu_industrial_area',
       'lu_industrial_share', 'lu_commercial_area', 'lu_commercial_share',
       'lu_residential_area', 'lu_residential_share', 'lu_retail_area',
       'lu_retail_share', 'lu_farmland_area', 'lu_farmland_share',
       'lu_farmyard_area', 'lu_farmyard_share', 'road_motorway_len',
       'road_trunk_len', 'road_primary_len', 'road_secondary_len',
       'road_tertiary_len', 'road_residential_len', 'fossil_pp_count',
       'geometry_x', 'date', 'no2_lag1', 'no2_neighbor_lag1', 'cloud_category',
       'LST_day_mean', 'landcover_2023', 'Shape_Leng', 'Shape_Area', 'ADM3_EN',
       'ADM3_PCODE', 'month'],
      dtype='object')

In [ ]:
full_df.head()

,geom_id,no2_mean,pop_sum_m,NTL_mean,road_len,road_share,poi_count,poi_share,lu_industrial_area,lu_industrial_share,...,no2_lag1,no2_neighbor_lag1,cloud_category,LST_day_mean,landcover_2023,Shape_Leng,Shape_Area,ADM3_EN,ADM3_PCODE,month
0,0,0.000051,969.68396,7.266073,5860.59401,0.000745,0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,12.0,0.538101,0.010351,Akaki Kality,ET140101,2023-01
1,0,0.000050,969.68396,10.372897,5860.59401,0.000745,0,0.0,0.0,0.0,...,0.000051,0.000038,0.0,25.130,12.0,0.538101,0.010351,Akaki Kality,ET140101,2023-01
2,0,0.000047,969.68396,1.124154,5860.59401,0.000745,0,0.0,0.0,0.0,...,0.000050,0.000052,0.0,15.588,12.0,0.538101,0.010351,Akaki Kality,ET140101,2023-01
3,0,0.000047,969.68396,0.727840,5860.59401,0.000745,0,0.0,0.0,0.0,...,0.000047,0.000050,0.0,30.670,12.0,0.538101,0.010351,Akaki Kality,ET140101,2023-01
4,0,0.000058,969.68396,3.964316,5860.59401,0.000745,0,0.0,0.0,0.0,...,0.000047,0.000044,0.0,30.990,12.0,0.538101,0.010351,Akaki Kality,ET140101,2023-01


#### Cell - Monthly Level

First, generate the cell level, monthly data and save as csv file.

In [ ]:
# features that need mean when aggregate from day to month level
time_agg_mean_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

monthly_cell = daily_cell_ori.groupby(['geom_id', 'month', 'ADM3_EN'])[time_agg_mean_feature].mean().reset_index()
monthly_cell.to_csv(DATA_PATH / "temp" / 'addis_monthly_cell.csv', index=False)

In [ ]:
# monthly_cell.head()

#### Sub-district - Daily Level

Second, generate the sub-district level, daily data and save as csv file.

In [ ]:
spatial_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]
space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

daily_adm3_sum = daily_cell_ori.groupby(['date', 'ADM3_EN'])[spatial_agg_sum_feature].sum().reset_index()
daily_adm3_avg = daily_cell_ori.groupby(['date', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()
daily_adm3 = daily_adm3_avg.merge(daily_adm3_sum, on=['date', 'ADM3_EN'], how='left')
daily_adm3.to_csv(DATA_PATH / "temp" / 'addis_daily_adm3.csv', index=False)

In [ ]:
# daily_adm3.head()

#### Sub-district - Monthly Level

Third, generate the sub-district level, monthly data and save as csv file.

In [ ]:
space_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

monthly_adm3_sum = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_sum_feature].sum().reset_index()
monthly_adm3_avg = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()

In [ ]:
monthly_adm3 = monthly_adm3_avg.merge(monthly_adm3_sum, on=['month', 'ADM3_EN'], how='left')
monthly_adm3.to_csv(DATA_PATH / "temp" / 'addis_monthly_adm3.csv', index=False)

In [ ]:
# monthly_adm3

### Baghdad

In [ ]:
import numpy as np
import pandas as pd
# Read the data
full_df = pd.read_parquet(DATA_PATH / "temp" / "full_baghdad_df.parquet", engine="pyarrow")
full_df['month'] = full_df['date'].dt.to_period('M')
full_df = full_df.rename(columns={'temp_mean': 'LST_day_mean'})   # unify to LST_day_mean

# Original dataset: daily / cell resolution
daily_cell_ori = full_df

In [ ]:
full_df.columns

Index(['geom_id', 'no2_mean', 'pop_sum_m', 'NTL_mean', 'road_len',
       'road_share', 'poi_count', 'poi_share', 'lu_industrial_area',
       'lu_industrial_share', 'lu_commercial_area', 'lu_commercial_share',
       'lu_residential_area', 'lu_residential_share', 'lu_retail_area',
       'lu_retail_share', 'lu_farmland_area', 'lu_farmland_share',
       'lu_farmyard_area', 'lu_farmyard_share', 'road_motorway_len',
       'road_trunk_len', 'road_primary_len', 'road_secondary_len',
       'road_tertiary_len', 'road_residential_len', 'fossil_pp_count', 'TCI',
       'geometry_x', 'date', 'no2_lag1', 'no2_neighbor_lag1', 'cloud_category',
       'LST_day_mean', 'landcover_2023', 'Shape_Leng', 'Shape_Area', 'ADM3_EN',
       'ADM3_PCODE', 'month'],
      dtype='object')

#### Cell - Monthly Level

First, generate the cell level, monthly data and save as csv file.

In [ ]:
# features that need mean when aggregate from day to month level
time_agg_mean_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'TCI',
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

monthly_cell = daily_cell_ori.groupby(['geom_id', 'month', 'ADM3_EN'])[time_agg_mean_feature].mean().reset_index()
monthly_cell.to_csv(DATA_PATH / "temp" / 'baghdad_monthly_cell.csv', index=False)

In [ ]:
# monthly_cell.head()

#### Sub-district - Daily Level
Second, generate the sub-district level, daily data and save as csv file.

In [ ]:
spatial_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'TCI',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]
space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

daily_adm3_sum = daily_cell_ori.groupby(['date', 'ADM3_EN'])[spatial_agg_sum_feature].sum().reset_index()
daily_adm3_avg = daily_cell_ori.groupby(['date', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()
daily_adm3 = daily_adm3_avg.merge(daily_adm3_sum, on=['date', 'ADM3_EN'], how='left')
daily_adm3.to_csv(DATA_PATH / "temp" / 'baghdad_daily_adm3.csv', index=False)

In [ ]:
daily_adm3

,date,ADM3_EN,LST_day_mean,no2_mean,no2_lag1,no2_neighbor_lag1,NTL_mean,TCI,pop_sum_m,road_len,...,lu_residential_area,lu_retail_area,lu_farmland_area,lu_farmyard_area,road_primary_len,road_motorway_len,road_trunk_len,road_secondary_len,road_tertiary_len,road_residential_len
0,2023-01-01,Abu Ghraib,12.420119,0.095418,0.000000,0.000000,16517.924950,2.796893e+06,5.111909e+05,2.712699e+06,...,2.436915e+07,0.0,3.297929e+07,8734.319149,72805.034518,134296.278767,0.000000,42553.429081,267884.154210,1.157555e+06
1,2023-01-01,Al-Fahama,12.771666,0.037629,0.000000,0.000000,7831.303788,3.661187e+07,1.019493e+06,1.771264e+06,...,3.925814e+07,0.0,1.072887e+06,0.000000,9374.771374,23504.337300,31207.293767,4882.810604,165507.886467,1.322054e+06
2,2023-01-01,Al-Jisr,13.350142,0.066570,0.000000,0.000000,6524.999007,5.499618e+06,2.537896e+05,6.784165e+05,...,9.367158e+06,0.0,1.893582e+07,0.000000,29253.292783,0.000000,31548.343818,14049.948014,52184.422521,3.835369e+05
3,2023-01-01,Al-Karrada Al-Sharqia,13.822724,0.043889,0.000000,0.000000,10284.304605,2.778583e+07,6.804654e+05,1.657440e+06,...,2.692580e+07,0.0,6.301526e+06,0.000000,86620.551798,32598.567966,3730.427599,40736.417691,118433.362443,8.482135e+05
4,2023-01-01,Al-Latifya,12.399308,0.072134,0.000000,0.000000,5038.490275,3.488123e+06,1.598628e+05,8.827485e+05,...,2.360183e+06,0.0,2.155193e+08,0.000000,7899.550833,58368.588112,39685.810012,40019.461397,59683.649226,2.846482e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,2024-12-31,Markaz Al-Karkh,13.878697,0.000000,0.021264,0.021154,8148.203962,2.332277e+05,1.772346e+05,7.375371e+05,...,1.005766e+07,0.0,0.000000e+00,0.000000,71372.128640,14098.977033,0.000000,50811.351119,31200.153262,2.529159e+05
15347,2024-12-31,Markaz Al-Mada'in,13.722056,0.000000,0.406985,0.404960,7029.665967,1.069760e+05,1.636998e+05,7.846149e+05,...,1.549490e+07,0.0,1.479552e+08,0.000000,22978.465260,0.000000,42400.937644,0.000000,42946.439662,4.669922e+05
15348,2024-12-31,Markaz Al-Mahmoudiya,13.583168,0.000000,0.034868,0.034775,1850.581081,1.531018e+05,1.072385e+05,4.749349e+05,...,8.522331e+03,0.0,3.965280e+07,0.000000,9762.885545,7638.868219,9296.516086,2541.127138,57374.549910,2.398848e+05
15349,2024-12-31,Markaz Al-Thawra,13.638146,0.000000,0.046052,0.046124,10100.558853,3.099584e+05,1.156935e+06,1.571223e+06,...,3.415564e+07,0.0,0.000000e+00,0.000000,109349.280838,12861.668398,0.000000,49075.556344,167318.888287,1.085157e+06


#### Sub-district - Monthly Level

Third, generate the sub-district level, monthly data and save as csv file.

In [ ]:
space_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'TCI',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

monthly_adm3_sum = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_sum_feature].sum().reset_index()
monthly_adm3_avg = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()

In [ ]:
monthly_adm3 = monthly_adm3_avg.merge(monthly_adm3_sum, on=['month', 'ADM3_EN'], how='left')
monthly_adm3.to_csv(DATA_PATH / "temp" / 'baghdad_monthly_adm3.csv', index=False)

In [ ]:
# monthly_adm3